# 風実装のためのテストプログラム

ライブラリ読み込み

In [1]:
import polars as pl
import numpy as np
from geopy.distance import geodesic
from geopy.point import Point
import math

# 初期方位角を計算する関数
from geographiclib.geodesic import Geodesic

風データの読み込み

In [2]:
year = 2016
month = 4

wind_data = pl.read_csv("ERA5/wind_datas/era5_testdata_E180W90S0W90_" + str(int(year)) + "_" + str(int(month)) + ".csv",encoding="shift-jis")

計算

In [20]:
# 風速情報と台風発電船の移動軌跡に最も近い地点を見つける関数
def find_nearest_wind_point(data, lat, lon):
    # 最も近い風速情報を取得
    nearest_point = data.with_columns(
        (pl.lit(lat) - pl.col('LAT')).abs().alias('lat_diff'),
        (pl.lit(lon) - pl.col('LON')).abs().alias('lon_diff')
    ).with_columns(
        (pl.col('lat_diff') + pl.col('lon_diff')).alias('total_diff')
    ).sort('total_diff').head(1)
    
    # 最も近い風速情報を取得
    u = nearest_point["U10_E+_W-[m/s]"][0]
    v = nearest_point["V10_N+_S-[m/s]"][0]

    #print("wind座標",nearest_point["LAT"][0],nearest_point["LON"][0])
    #print("現在地から1番近いポイント＝",nearest_point)

    return u, v

# 台風発電船が風から得るエネルギーを計算する関数
def calculate_energy(u, v, angle_difference, time_interval):
    # 風速の計算
    wind_speed = math.sqrt(u ** 2 + v ** 2)

    # 風速と角度差を考慮してエネルギーを計算
    if abs(angle_difference) <= 80 or abs(angle_difference) >= 280:
        # 追い風　単位は[Wh]になるように調整
        #C = 10**5
        energy = (wind_speed**2) * time_interval
        #print("追い風")
    elif abs(angle_difference) <= 100 or abs(angle_difference) >= 260:
        # 横風　単位は[Wh]になるように調整
        #C = 10**5
        energy = (wind_speed**2) * time_interval
        #print("横風")
    else:
        # 向かい風　単位は[Wh]になるように調整
        #C = 10**5
        energy = -(wind_speed**2) * time_interval
        #print("向かい風")
    
    return energy



def calculate_initial_bearing(start_coord, end_coord):
    # Geodesicクラスのインスタンスを作成
    geod = Geodesic.WGS84

    
    # 始点と終点の緯度と経度
    lat1, lon1 = start_coord[0], start_coord[1]
    lat2, lon2 = end_coord[0], end_coord[1]
    
    # geod.Inverse()を使用して始点から終点までの地理的な情報を取得
    # 結果として辞書が返され、辞書のキー 'azi1' は初期方位角を含む
    result = geod.Inverse(lat1, lon1, lat2, lon2)
    
    # 初期方位角を取得（北を0度とし、時計回りに測定）
    initial_bearing = result['azi1']
    
    return initial_bearing


# 移動軌跡と風速情報からエネルギーを計算するプログラム
def calculate_trajectory_energy(data, start_coord, end_coord, timestep):

    # 初期方位角を計算
    initial_bearing = calculate_initial_bearing(start_coord, end_coord)
    
    # 始点と終点を Point オブジェクトに変換
    start_point = Point(start_coord)
    end_point = Point(end_coord)
    
    # 大圏距離を計算
    geo_dist = geodesic(start_point, end_point)
    total_distance = geo_dist.kilometers
    
    # 時間間隔の設定
    time_interval = timestep / (2 * timestep)
    
    # 初期化
    current_position = start_point
    current_bearing = initial_bearing
    total_energy = 0
    
    # 時間間隔ごとの移動
    for i in range(int(2 * timestep)):
        # 現在の時刻
        time = i * time_interval
        
        # 指定された時間での移動距離を計算
        distance_travelled = (total_distance / timestep) * time
        
        # 現在の位置を計算
        current_position = geodesic(kilometers=distance_travelled).destination(
            point=start_point, bearing=current_bearing
        )
        
        # 現在の方位角を更新
        current_bearing = calculate_initial_bearing(current_position, end_coord)
        
        # 現在の位置の緯度経度
        current_lat, current_lon = current_position.latitude, current_position.longitude
        
        # 風速データを取得
        # ここでは風速データを取得するための関数 find_nearest_wind_point を使用します。
        # この関数は data から緯度経度に最も近い風速データを取得するものです。
        u, v = find_nearest_wind_point(data, current_lat, current_lon)
        
        # 風向角度を計算（北を 0 度として時計回りに増加）
        wind_angle = math.degrees(math.atan2(v, u))
        # 北を 0 度として時計回りに測定するように調整
        wind_angle = (360 - wind_angle + 90) % 360 
        
        # 船の方位角と風向の角度差を計算
        if current_bearing < 0:
            current_bearing += 360
        
        # 船の方位から見て風がどの方向に向かって吹いているのかを計算している。（必要に応じて吹いて来ている方向に直す）
        angle_difference = (wind_angle - current_bearing + 360) % 360
        
        # エネルギーを計算
        energy = calculate_energy(u, v, angle_difference, time_interval)
        
        # エネルギーを累積
        total_energy += energy

        #print("ship座標",current_lat,current_lon)
        #print("風速",u,v)
        #print("進路",current_bearing)
        #print("風向",wind_angle)
        #print("角度差",angle_difference)
        #print("エネルギー",energy)

    
    return total_energy

# 開始座標、終了座標、時間間隔
# 始点と終点の座標
start_coord = (40.0, 135.0)
end_coord = (39.0, 137.0)
timestep = 6

# プログラムを実行してエネルギーを計算
data = wind_data
total_energy = calculate_trajectory_energy(data, start_coord, end_coord, timestep)

#print(end_coord)

#print(f'点Sが風から得たエネルギー: {total_energy}')


エネルギー 1.2287686166519614
エネルギー 1.225699396636234
エネルギー 0.9017838101403328
エネルギー 0.8751319062565994
エネルギー 0.8523889363687848
エネルギー 0.6361665411174088
エネルギー 0.5878675799790961
エネルギー 0.5496676396279278
エネルギー 0.3359458151459608
エネルギー 0.3101534528723749
エネルギー 0.311677719159496
エネルギー 0.19902143307592857
点Sが風から得たエネルギー: 8.014272847032105


In [22]:
import numpy as np
from turtle import distance
import csv
import math
import geopy as gp
import folium as fl
import matplotlib.pyplot as plt
import japanize_matplotlib
import os
from geopy.distance import geodesic
from geopy.point import Point
from geographiclib.geodesic import Geodesic
from datetime import datetime, timedelta, timezone
from dateutil import tz
import random
from matplotlib import pyplot as pyp

In [25]:
year = 2021
time_step = 10000

UTC = timezone(timedelta(hours =+ 0),"UTC")
datetime_1_1 = datetime(year,1,1,0,0,0,tzinfo = tz.gettz("UTC"))
current_time = int(datetime_1_1.timestamp())
#current_timeから月の取得
month = datetime_1_1.month

print(month)

time_step_unix = 3600 * time_step

#timestep後の時刻の取得
current_time = current_time + time_step_unix



date = datetime.fromtimestamp(current_time,UTC)
month = date.month
print(date)
print(month)



1
2022-02-21 16:00:00+00:00
2
